In [46]:
# Benders decomposition for Fixed Charge Transportation Problem

import gurobipy as gp
from gurobipy import *
import numpy as np


In [55]:
capacity = [5,8,7]
demand = [3,6,6,5]

locations = len(capacity)
customers = len(demand)

varcost = [[4,5,2,7],[5,8,6,2],[8,9,4,3]]
fixcost = [[2,10,1,5],[7,2,7,8],[4,0,3,9]]

In [56]:
# add 1 locations with very large capacity and very high cost 
# => later force this location delivered to all customers to make solution always feasible
capacity.append(sum(demand))
varcost.append([np.max(fixcost)*100 for i in range(customers)])
fixcost.append([np.max(fixcost)*100 for i in range(customers)])
locations += 1

bigM = [[min(capacity[i], demand[j]) for j in range(customers)] for i in range(locations)]

In [57]:
np.max(fixcost)*10

10000

In [58]:
add_cost = fixcost[locations-1][0] * customers
add_cost

4000

In [61]:
# Define Benders subproblem
        
def subproblem(model, where):
    if where == GRB.Callback.MIPSOL:
        v_y = model.cbGetSolution(model._y)
        #print('\nCurrent y: ', v_y)
        
        LB = model.cbGetSolution(model._z)
        print('Current LB: ', LB, '\n')
        print(model.cbGet(GRB.Callback.MIPSOL_OBJBND))
        print(model.cbGet(GRB.Callback.MIPSOL_OBJBST))
        
        
        bsp = gp.Model("Subproblem") 
        #variables
        x = {}
        for i in range(locations):
            for j in range(customers):
                x[i,j] = bsp.addVar(obj=varcost[i][j])
        
        #demand constraint
        demand_constr = {} 
        for j in range(customers):
            demand_constr[j] = bsp.addConstr(sum(x[i,j] for i in range(locations)) >= demand[j])
        
        #capacity constraint
        cap_constr = {}
        for i in range(locations):
            cap_constr[i] = bsp.addConstr(sum(x[i,j] for j in range(customers)) <= capacity[i])
        
        #logical constraint
        log_constr = {}
        for i in range(locations):
            for j in range(customers):
                log_constr[i,j] = bsp.addConstr(x[i,j] <= v_y[i,j]*bigM[i][j])
        
        bsp.optimize()
        
        fc = 0
        for i in range(locations):
            for j in range(customers):
                if x[i,j].x > 0: fc+=fixcost[i][j]
                    
        #update ub and lb
        if bsp.objVal + fc > model.cbGet(GRB.Callback.MIPSOL_OBJBND) :
            v = np.zeros(customers) #dual of demand constraints
            for j in range(customers):
                v[j] = demand_constr[j].pi #get dual value
            
            w = np.zeros((locations,customers)) #dual of logical constraints
            for i in range(locations):
                for j in range(customers):
                    w[i,j] = log_constr[i,j].pi
            
            u = np.zeros(locations) #dual of capacity constraints
            for i in range(locations):
                u[i] = cap_constr[i].pi
            
            cm = np.zeros((locations,customers)) #coefficient of y in master problem
            for i in range(locations):
                for j in range(customers):
                    cm[i,j] = fixcost[i][j] + bigM[i][j]*w[i,j]
            
            #print(bsp.objVal)
            #print(cm)
            #print(v)
            #print(w)
            #print(u)
            

            model.cbLazy(model._z >= sum(demand[j] * v[j] for j in range(customers)) +   
                         sum(capacity[i] * u[i] for i in range(locations)) + 
                         sum(cm[i,j] * model._y[i,j] for i in range(locations) for j in range(customers)))
        
        # save result of x from subproblem
        global transport
        for i in range(locations):
            for j in range(customers):
                transport[i,j] = x[i,j].x
        print([transport[locations-1][j] for j in range(customers)])
        #print(bsp.objVal)

        bsp.dispose
                       

In [62]:
transport = np.zeros((locations, customers)) # to save result of x from subproblem
# Masterproblem
                     
m = gp.Model("Benders Fixed Charge Transportation Masterproblem")

y = {}
z = m.addVar(obj=1)                                                                           
for i in range(locations):
    for j in range(customers):
        y[i,j] = m.addVar(vtype=GRB.BINARY)

# force imaginary location to deliver to all customers to guarantee solution always feasible
for j in range(customers):
    m.addConstr(y[locations-1,j] == 1)
    m.addConstr(sum(y[i,j] for i in range(locations-1)) >= 1)



m.Params.lazyConstraints = 1
m._y = y
m._z = z

m.optimize(subproblem)

#calculate the added fixed cost due to added location to calculate true objective value
#never actualy deliver any quantity from this location => added varcost = 0

print("Objective: " + str(m.objVal - add_cost))

import pandas as pd
print("Solution:")
index = ['Source ' + str(x) for x in range(locations)]
columns = ['Customer ' + str(x) for x in range(customers)]

solution = pd.DataFrame(index=index, columns=columns)

for i in range(locations):
    for j in range(customers):
        solution.iloc[i,j] = transport[i,j]
for i in range(locations):    
    print([y[i,j].x for j in range(customers)] )        
print(solution)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 8 rows, 17 columns and 16 nonzeros
Model fingerprint: 0xe3d56334
Variable types: 1 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Current LB:  0.0 

-1e+100
0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 24 rows, 16 columns and 48 nonzeros
Model fingerprint: 0xbbfbf655
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 2e+01]
Presolve removed 17 rows and 0 columns
Presolve time: 0.01s
Presolved: 7 rows, 16 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.150000e+01   0.000000e+00 

       4    1.0500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds
Optimal objective  1.050000000e+02
[0.0, 0.0, 0.0, 0.0]
H    0     0                    4125.0000000    0.00000   100%     -    0s
     0     0 4058.23699    0    1 4125.00000 4058.23699  1.62%     -    0s
Current LB:  4058.9999999999995 

4058.2369942196524
4058.9999999999995
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 24 rows, 16 columns and 48 nonzeros
Model fingerprint: 0x7b66e2ee
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 2e+01]
Presolve removed 20 rows and 9 columns
Presolve time: 0.01s
Presolved: 4 rows, 7 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2000000e+01   6.000000e+00   0.000000e+00      0s
Extra one simplex iteration after uncrush
       5    3.0650000e+03   0.000000e+00 

In [69]:
file = open('43.txt','r')
data = file.readlines()

[locations, customers] = [int(x) for x in data[0].split(' ')]
capacity = [int(data[1].split(' ')[i]) for i in range(locations) ]
demand = [int(data[2].split(' ')[j]) for j in range(customers) ]

fixcost = [ [ int(data[i+3].split('   ')[j]) for j in range(customers) ] for i in range(locations)]
varcost = [ [ int(data[i+3+locations+1].split('    ')[j]) for j in range(customers) ] for i in range(locations)]

print(locations)
print(customers)
print(capacity)
print(demand)
varcost

15
15
[20, 20, 20, 18, 18, 17, 17, 10, 10, 9, 9, 4, 4, 3, 3]
[20, 19, 19, 18, 17, 16, 16, 12, 11, 11, 7, 5, 5, 5, 1]


[[79, 68, 70, 25, 17, 65, 75, 67, 58, 61, 49, 34, 59, 33, 46],
 [39, 47, 43, 18, 55, 44, 41, 17, 73, 62, 56, 53, 41, 78, 80],
 [54, 43, 49, 74, 68, 55, 34, 74, 36, 73, 29, 75, 18, 20, 33],
 [24, 49, 19, 38, 50, 43, 56, 47, 79, 71, 69, 73, 78, 78, 24],
 [47, 18, 46, 25, 30, 69, 59, 68, 72, 49, 21, 39, 43, 36, 65],
 [20, 40, 26, 25, 28, 32, 39, 60, 77, 23, 53, 75, 73, 42, 60],
 [35, 81, 55, 25, 56, 39, 48, 78, 81, 50, 75, 38, 54, 39, 35],
 [77, 69, 61, 67, 64, 39, 48, 71, 52, 53, 58, 66, 26, 68, 45],
 [49, 20, 81, 57, 40, 73, 32, 19, 27, 30, 68, 22, 25, 53, 58],
 [46, 38, 35, 74, 73, 68, 58, 55, 29, 31, 32, 43, 34, 41, 30],
 [56, 40, 80, 75, 79, 29, 39, 35, 23, 63, 60, 16, 56, 35, 65],
 [76, 56, 68, 52, 42, 23, 18, 33, 31, 73, 19, 20, 22, 79, 74],
 [43, 59, 78, 16, 19, 44, 56, 43, 57, 16, 42, 37, 72, 29, 47],
 [45, 44, 54, 38, 79, 44, 67, 26, 22, 47, 38, 57, 34, 45, 31],
 [50, 16, 67, 80, 76, 31, 48, 43, 71, 44, 64, 21, 72, 40, 22]]

In [70]:
add_cost = fixcost[locations-1][0] * customers
add_cost

4275

In [71]:
# add 1 locations with very large capacity and very high cost 
# => later force this location delivered to all customers to make solution always feasible
capacity.append(sum(demand))
#M = np.max(varcost)*np.max(fixcost)*10
varcost.append([np.max(fixcost)*100 for i in range(customers)])
fixcost.append([np.max(fixcost)*100 for i in range(customers)])
locations += 1
print()

bigM = [[min(capacity[i], demand[j]) for j in range(customers)] for i in range(locations)]

In [72]:
transport = np.zeros((locations, customers)) # to save result of x from subproblem
# Masterproblem
                     
m = gp.Model("Benders Fixed Charge Transportation Masterproblem")

y = {}
z = m.addVar(obj=1)                                                                           
for i in range(locations):
    for j in range(customers):
        y[i,j] = m.addVar(vtype=GRB.BINARY)

# force imaginary location to deliver to all customers to guarantee solution always feasible
for j in range(customers):
    m.addConstr(y[locations-1,j] == 1)
    m.addConstr(sum(y[i,j] for i in range(locations-1)) >= 1)

m.Params.lazyConstraints = 1
m._y = y
m._z = z

m.Params.MIPGap = 0.05
m.optimize(subproblem)

#calculate the added fixed cost due to added location to calculate true objective value
#never actualy deliver any quantity from this location => added varcost = 0


print("Objective: " + str(m.objVal - add_cost))

print("Solution:")
import pandas as pd
index = ['Source ' + str(x) for x in range(locations)]
columns = ['Customer ' + str(x) for x in range(customers)]

solution = pd.DataFrame(index=index, columns=columns)

#for i in range(locations):
#    for j in range(customers):
#        solution.iloc[i,j] = transport[i,j]
for i in range(locations):    
    print([y[locations-1,j].x for j in range(customers)] )         
#print(solution)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 30 rows, 241 columns and 240 nonzeros
Model fingerprint: 0xf79628b7
Variable types: 1 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Current LB:  0.0 

-1e+100
0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 271 rows, 240 columns and 720 nonzeros
Model fingerprint: 0xb6303d05
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 241 rows and 0 columns
Presolve time: 0.02s
Presolved: 30 rows, 240 columns, 465 nonzeros

Iteration    

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 252 rows and 200 columns
Presolve time: 0.02s
Presolved: 19 rows, 40 columns, 67 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0183300e+05   7.150000e+01   0.000000e+00      0s
Extra 2 simplex iterations after uncrush
      26    1.1049750e+06   0.000000e+00   0.000000e+00      0s

Solved in 26 iterations and 0.03 seconds
Optimal objective  1.104975000e+06
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 1.0, 5.0, 0.0, 0.0]
Current LB:  0.0 

0.0
0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 271 rows, 240 columns and 720 nonzeros
Model fingerprint: 0x210e1daf
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve remo


Solved in 32 iterations and 0.02 seconds
Optimal objective  8.794768000e+06
[3.0, 9.0, 10.0, 18.0, 7.0, 0.0, 13.0, 9.0, 7.0, 11.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Current LB:  10210385.000000002 

116603.4574402248
10210385.000000002
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 271 rows, 240 columns and 720 nonzeros
Model fingerprint: 0x06186d3a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 262 rows and 205 columns
Presolve time: 0.01s
Presolved: 9 rows, 35 columns, 45 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.6916846e+06   1.201250e+01   0.000000e+00      0s
Extra 24 simplex iterations after uncrush
      32    8.6949040e+06   0.000000e+00   0.000000e+00      0s

Solved in 32 iterations and 0.03 seconds
Optimal objective  8.694904000e+06
[3.0, 9.0, 10.0, 18.0, 7.0, 0.0, 13.0, 9.0, 7.0,

Optimize a model with 271 rows, 240 columns and 720 nonzeros
Model fingerprint: 0x91d2bbd4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 248 rows and 182 columns
Presolve time: 0.02s
Presolved: 23 rows, 58 columns, 98 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5985495e+06   6.200600e+01   0.000000e+00      0s
      22    1.6052370e+06   0.000000e+00   0.000000e+00      0s

Solved in 22 iterations and 0.03 seconds
Optimal objective  1.605237000e+06
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Current LB:  1925619.0000000002 

117906.36079759957
1925619.0000000002
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 271 rows, 240 columns and 720 nonzeros
Model fingerprint: 0x5d94f947
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [

Optimize a model with 271 rows, 240 columns and 720 nonzeros
Model fingerprint: 0x6474b7f1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 247 rows and 183 columns
Presolve time: 0.01s
Presolved: 24 rows, 57 columns, 98 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0992355e+06   6.950600e+01   0.000000e+00      0s
      20    1.4051060e+06   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.02 seconds
Optimal objective  1.405106000e+06
[0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Current LB:  1724442.9999999981 

117906.36079759957
1724442.9999999981
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 271 rows, 240 columns and 720 nonzeros
Model fingerprint: 0x536da5be
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [

      31    1.3055050e+06   0.000000e+00   0.000000e+00      0s

Solved in 31 iterations and 0.02 seconds
Optimal objective  1.305505000e+06
[0.0, 1.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Current LB:  1224907.9999999981 

118524.89147227388
1224907.9999999981
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 271 rows, 240 columns and 720 nonzeros
Model fingerprint: 0x0cafe2e7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 247 rows and 181 columns
Presolve time: 0.01s
Presolved: 24 rows, 59 columns, 103 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2001657e+06   5.650650e+01   0.000000e+00      0s
      27    1.2057140e+06   0.000000e+00   0.000000e+00      0s

Solved in 27 iterations and 0.02 seconds
Optimal objective  1.205714000e+06
[0.0, 0.0, 10.0, 0.0, 0.0,

Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 248 rows and 179 columns
Presolve time: 0.01s
Presolved: 23 rows, 61 columns, 103 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4986474e+06   6.700800e+01   0.000000e+00      0s
      25    2.4037530e+06   0.000000e+00   0.000000e+00      0s

Solved in 25 iterations and 0.02 seconds
Optimal objective  2.403753000e+06
[0.0, 0.0, 0.0, 0.0, 7.0, 0.0, 7.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Current LB:  119964.0 

118836.54608693291
119964.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 271 rows, 240 columns and 720 nonzeros
Model fingerprint: 0xc52bb227
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 244 rows an

Optimize a model with 271 rows, 240 columns and 720 nonzeros
Model fingerprint: 0xaacebdd5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 248 rows and 182 columns
Presolve time: 0.01s
Presolved: 23 rows, 58 columns, 97 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0013155e+05   6.750500e+01   0.000000e+00      0s
      27    3.8017860e+06   0.000000e+00   0.000000e+00      0s

Solved in 27 iterations and 0.02 seconds
Optimal objective  3.801786000e+06
[6.0, 1.0, 19.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Current LB:  120210.0 

118858.43261602198
120210.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 271 rows, 240 columns and 720 nonzeros
Model fingerprint: 0x342ff878
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+05]
  Bo

In [24]:
k = 1
mvars = m.getVars()
y0 = np.zeros((locations, customers))
for i in range(locations):
    for j in range(customers):
        y0[i,j] = mvars[k].x
        k+=1
y0

array([[-0., -0., -0., -0., -0.,  1., -0., -0.,  1., -0., -0., -0., -0.,
        -0., -0.],
       [-0.,  1., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
        -0., -0.],
       [-0., -0.,  1., -0.,  1., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0.],
       [-0., -0., -0.,  1., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0.],
       [-0., -0.,  1., -0., -0., -0., -0., -0., -0., -0., -0.,  1., -0.,
        -0., -0.],
       [ 1., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0.],
       [ 1., -0., -0.,  1., -0.,  1.,  1., -0., -0., -0., -0., -0., -0.,
        -0., -0.],
       [-0., -0., -0., -0., -0., -0.,  1.,  1., -0., -0., -0., -0., -0.,
        -0.,  1.],
       [-0., -0., -0., -0.,  1., -0., -0.,  1., -0., -0., -0., -0., -0.,
        -0., -0.],
       [-0., -0.,  1., -0., -0., -0., -0., -0., -0., -0.,  1., -0., -0.,
        -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0.,  1., -0., -0., -0.,
       

In [25]:
model0 = gp.Model()
x0 = {}
for i in range(locations):
    for j in range(customers):
        x0[i,j] = model0.addVar(obj=varcost[i][j])

#demand constraint
demand_constr = {} 
for j in range(customers):
    demand_constr[j] = model0.addConstr(sum(x0[i,j] for i in range(locations)) >= demand[j])
        
#capacity constraint
cap_constr = {}
for i in range(locations):
    cap_constr[i] = model0.addConstr(sum(x0[i,j] for j in range(customers)) <= capacity[i])

#logical constraint
log_constr = {}
for i in range(locations):
    for j in range(customers):
        log_constr[i,j] = model0.addConstr(x0[i,j] <= y0[i,j] * min(capacity[i], demand[j]))

model0.optimize()

print("Objective: " + str(model0.objVal))

print("Solution:")
import pandas as pd
index = ['Source ' + str(x) for x in range(locations)]
columns = ['Customer ' + str(x) for x in range(customers)]

solution = pd.DataFrame(index=index, columns=columns)

for i in range(locations):
    for j in range(customers):
        solution.iloc[i,j] = x0[i,j].x

print(solution)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 287 rows, 255 columns and 765 nonzeros
Model fingerprint: 0x0c103ea3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 272 rows and 226 columns
Presolve time: 0.01s
Presolved: 15 rows, 29 columns, 43 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9975850e+06   6.150000e+01   0.000000e+00      0s
Extra 6 simplex iterations after uncrush
      20    7.0006660e+06   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.03 seconds
Optimal objective  7.000666000e+06
Objective: 7000666.0
Solution:
           Customer 0  Customer 1  Customer 2  Customer 3  Customer 4  \
Source 0            0           0           0           0           0   
Source 1            0          19           0           0           0   
Source 2            0     

In [29]:
model0.objVal+fc

7007716.0

In [27]:
fc = 0
for i in range(locations-1):
    for j in range(customers):
        if x0[i,j].x > 0: 
            fc+=fixcost[i][j]

In [28]:
fc

7050